<a href="https://colab.research.google.com/github/MA541121/EconomiaIII/blob/main/streamlit_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install streamlit pyngrok -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 104.7 MB/s eta 0:00:00


In [3]:
import streamlit as st
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.decomposition import PCA

In [4]:

# --- Configuración de la app ---
st.set_page_config(page_title="Modelos No Supervisados", layout="wide")
st.title("🌸 Análisis de Modelos No Supervisados con Iris Dataset")

2025-11-10 02:59:33.525 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 02:59:33.527 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 02:59:33.729 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-10 02:59:33.730 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 02:59:33.731 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [5]:
# --- Cargar datos ---
st.header("1️⃣ Dataset Iris")
iris = sns.load_dataset('iris')
st.dataframe(iris.head())

2025-11-10 03:03:25.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:25.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:25.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:25.989 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:25.990 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:25.991 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [6]:
st.write("### Descripción estadística")
st.write(iris.describe())

2025-11-10 03:03:47.682 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:47.683 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:47.685 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:47.702 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:47.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:03:47.704 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [7]:
# --- Distribución por especie ---
species_count = iris["species"].value_counts().reset_index()
species_count.columns = ['species', 'count']
fig = px.bar(species_count, x="species", y="count", title="Distribución de especies")
st.plotly_chart(fig, use_container_width=True)

2025-11-10 03:04:52.509 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-11-10 03:04:52.531 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:04:52.535 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:04:52.539 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:04:52.541 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:04:52.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [8]:
# --- Gráfica de caja ---
st.header("2️⃣ Análisis exploratorio")
fig = go.Figure()
for feature in iris.columns[:-1]:
    fig.add_trace(go.Box(y=iris[feature], x=iris["species"], name=feature))
fig.update_layout(title="Gráfica de caja por especie")
st.plotly_chart(fig, use_container_width=True)

2025-11-10 03:05:15.691 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:15.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:15.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:15.728 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-11-10 03:05:15.738 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:15.740 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:15.745 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [9]:
# --- Mapa de calor de correlaciones ---
corr_matrix = iris.drop('species', axis=1).corr()
fig = px.imshow(corr_matrix, text_auto=True, title="Mapa de correlación")
st.plotly_chart(fig, use_container_width=True)

2025-11-10 03:05:47.589 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-11-10 03:05:47.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:47.592 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:47.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:47.594 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:05:47.595 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [22]:
# --- Selección de modelo ---
st.header("3️⃣ Modelos de Clustering")
X = iris.drop('species', axis=1)
modelo = st.selectbox("Selecciona un modelo:", ["KMeans", "Clustering Jerárquico", "DBSCAN", "PCA"])

# --- KMeans ---
if modelo == "KMeans":
    k = st.slider("Número de clusters (k):", 2, 10, 3)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    iris["cluster"] = kmeans.fit_predict(X).astype(str)

    st.subheader("Resultado del clustering")
    fig = px.scatter(iris, x='sepal_length', y='petal_length', color='cluster',
                     title=f"KMeans con k={k}")
    st.plotly_chart(fig, use_container_width=True)

    silhouette = silhouette_score(X, kmeans.labels_)
    st.metric(label="Silhouette Score", value=f"{silhouette:.3f}")

# --- Clustering jerárquico ---
elif modelo == "Clustering Jerárquico":
    metodo = st.selectbox("Método de enlace:", ["single", "complete", "average"])
    max_d = st.slider("Máxima distancia:", 1, 10, 3)
    Z = linkage(X, metodo)
    clusters = fcluster(Z, max_d, criterion='distance')
    iris["cluster"] = clusters.astype(str)

    st.subheader("Dendrograma")
    fig, ax = plt.subplots(figsize=(10, 5))
    dendrogram(Z)
    st.pyplot(fig)

    st.subheader("Visualización por clusters")
    fig = px.scatter(iris, x='sepal_length', y='petal_length', color='cluster')
    st.plotly_chart(fig, use_container_width=True)

# --- DBSCAN ---
elif modelo == "DBSCAN":
    eps = st.slider("eps (radio máximo):", 0.1, 2.0, 0.5, 0.1)
    min_samples = st.slider("min_samples:", 1, 10, 5)
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(X)
    iris["cluster"] = labels.astype(str)
    st.subheader("Resultado del clustering con DBSCAN")
    fig = px.scatter(iris, x='sepal_length', y='petal_length', color='cluster')
    st.plotly_chart(fig, use_container_width=True)

# --- PCA ---
elif modelo == "PCA":
    st.subheader("Reducción de dimensionalidad con PCA")
    pca = PCA(n_components=2)
    X_pca = pd.DataFrame(pca.fit_transform(X), columns=['PCA1', 'PCA2'])
    X_pca['species'] = iris['species']
    fig = px.scatter(X_pca, x='PCA1', y='PCA2', color='species', title="PCA")
    st.plotly_chart(fig, use_container_width=True)

    st.write("### Varianza explicada")
    st.write(pca.explained_variance_ratio_)

2025-11-10 03:13:59.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:13:59.168 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:13:59.170 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:13:59.175 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:13:59.179 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:13:59.183 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:13:59.184 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-10 03:13:59.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar